## Checking autograd forces against analytical forces for Embedded Atom model
This notebook compares the forces computed with automatic differentiation (with respect to bond vectors, not atom coordinates) against the analytical forces for the [Al99 Embedded Atom potential](https://www.ctcms.nist.gov/potentials/entry/1999--Mishin-Y-Farkas-D-Mehl-M-J-Papaconstantopoulos-D-A--Al/), as implemented by [ASE](https://wiki.fysik.dtu.dk/ase/ase/atoms.html).

First, fetch the EAM potential data from interatomic potentials repository:

In [1]:
!curl https://www.ctcms.nist.gov/potentials/Download/1999--Mishin-Y-Farkas-D-Mehl-M-J-Papaconstantopoulos-D-A--Al/2/Al99.eam.alloy -o Al99.eam.alloy

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  762k  100  762k    0     0   181k      0  0:00:04  0:00:04 --:--:--  181k


Gradient checking is best done in 64-bit floating point precision:

In [2]:
from plum import activate_autoreload
activate_autoreload()

from pathlib import Path

import torch
from ase.calculators.eam import EAM
from ase.build import bulk

from nfflr.data import graph
from nfflr.data.atoms import Atoms
from nfflr.models.classical.eam import TorchEAM

torch.set_default_dtype(torch.float64)

We set up a small FCC aluminum system and add a small amount of jitter to the atomic coordinates:

In [3]:
# set up a small bulk Aluminum calculation
a = 4.05  
al_ase = bulk("Al", "fcc", a) * [4, 4, 4]
al_ase.rattle(stdev=0.05)
al_ase.wrap()
ase_eam = EAM(potential="Al99.eam.alloy")
al_ase.set_calculator(ase_eam)

# set up pytorch version
al = Atoms(al_ase.get_cell().array, al_ase.get_scaled_positions(), al_ase.numbers)
torch_eam = TorchEAM("Al99.eam.alloy", dtype=torch.float64)

The ASE implementation computes the forces using the analytical gradients of the EAM spline components, while the pytorch implementation uses automatic differentiation to compute the gradient of the energy with respect to the bond displacement vectors. A [sum reduction ](https://github.com/usnistgov/nfflr/blob/2026152caa6beab2fa0dcf066288223726e78215/nfflr/models/utils.py#L43-L51) is used to aggregate these into the force components on individual atoms.

Both the energies and the force components on all the atoms match to within floating point precision:

In [4]:
# construct radius graph matching ASE cutoff
g = graph.periodic_radius_graph(al, r=torch_eam.data.cutoff, dtype=torch.float64)

# evaluate energies and forces with pytorch implementation
e_dgl, force_dgl = torch_eam(g)
e_dgl, force_dgl = e_dgl.detach(), force_dgl.detach()

# evaluate energy and forces with ASE
e_ase = al_ase.get_potential_energy()
force_ase = al_ase.get_forces()

print(f"{al_ase.get_potential_energy()=}")
print(f"{e_dgl.detach().item()=}")
print(f"potential energy matches: {np.isclose(e_ase, e_dgl.item())}")
print(f"force components match? : {np.isclose(force_ase, force_dgl).all()}")

/Users/bld/.pyenv/versions/3.10.9/envs/nfflr/lib/python3.10/site-packages/dgl/backend/pytorch/tensor.py:445: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


al_ase.get_potential_energy()=-214.15885773313198
e_dgl.detach().item()=-214.15885773313158
potential energy matches: True
force components match? : True
